In [3]:
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from itertools import groupby
import pandas as pd
import pickle
import numpy as np
import math
import re
#Считываем файл, поправляя кодировку
Data = pd.read_csv('D:/Downloads/german_labeled.csv', sep=' ==> ', header=None, encoding = 'utf-8 sig')
#Получаем список тегов
Tags = [el for el, _ in groupby(sorted([str(tag) for tag in Data[4]]))]

#Инициализируем словарь предложений по тегам
Sentences_by_tag = {}
i=0
for Tag in Tags:    
    Sentences_by_tag[Tag] = i
    i=i+1

    
#Записываем все предложения по каждому тегу в словарь    
Sentences = ''
for Unique_tag in Sentences_by_tag.keys():
    i = 0
    Sentences_by_tag[Unique_tag] = []
    for Tag_to_compare in [str(tag) for tag in Data[4]]:         
        if Tag_to_compare == Unique_tag:
            Sentences += ' '+ str(Data[3][i])
        i=i+1
    Sentences_by_tag[Unique_tag] = Sentences
    Sentences = ''
    
#Получаем все предложения, отсортированные по тегам  
Documents = []
for Unique_tag in Sentences_by_tag.keys():
    Documents.append(Sentences_by_tag[Unique_tag])
    

#Получаем tf\idf матрицу с помощью sklearn
vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(Documents)
Tf_idf=pd.DataFrame(matrix.toarray(), columns=vectorizer.get_feature_names(), index= Sentences_by_tag.keys())


D:\Conda3\lib\site-packages\ipykernel_launcher.py:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  # Remove the CWD from sys.path while we load stuff.


In [4]:
#Функция, превращающая предложение в список чисел в соответствии со схемой
def To_vector (Sentence):
    stolbec = np.zeros(83219)
    word_list = vectorizer.get_feature_names()
    if isinstance(Sentence, str):
        splittedSentence = re.split(r"[,.!?:;\"\'\(\)\%\=\#\&\@\$\-\*\+\>\<\/\ \s]", Sentence)
        while ("") in splittedSentence:
            splittedSentence.remove("")

        for word in splittedSentence:
            try:
                position = word_list.index(word.lower())                
                stolbec[position] += 0.5
            except ValueError:
                continue
                
        return(np.matmul(Tf_idf,stolbec))
    else:
        return(np.matmul(Tf_idf,np.zeros(83219)))

In [5]:
Training_data = []
for i in zip(Data[4],Data[3]):
    Training_data.append(i)

Test_data = Training_data[109743:]
Training_data = Training_data[:109743]

In [6]:
#Превращаем валидационную выборку в вектора(работает ~40 минут(осторожно))
Test = []
for pair in Test_data:    
    Test.append([pair[0], To_vector(pair[1])]) 
Test

[]

In [12]:
#Запись в файл
data = {
    'test': Test
}
with open('path_to_file.pkl', 'wb') as output_file:
    pickle.dump(data, output_file, protocol=pickle.HIGHEST_PROTOCOL)
    


In [13]:
#Считывание из файла
with open('path_to_file.pkl', 'rb') as input_file:
    data = pickle.load(input_file)
Ntcn = data['test']

In [ ]:
#Пока не работает\не нужно
#a = np.array(Sentences_by_tag.keys())
#b = np.array(Sentences_by_tag.values())
#c = np.array(x.loc["0"])
#model = svm.SVC(kernel='rbf', C=1,gamma="auto")
#model.fit(matrix.toarray(), a.reshape(-1,1))
#model.predict(c.reshape(1,-1))
#
#Filtered_words = []
#for word in Documents:
#    Filtered_words+=re.split(r"[,.!?:;\"\'\(\)\%\=\#\&\@\$\-\*\+\>\<\/\ 1234567890\s]", str(word))
#Filtered_words=filter(None, Filtered_words)
#Unique_words = [el for el, _ in groupby(sorted(Filtered_words))]
#
#             ind = x.columns.index(word)
#             for feature in x.columns:
#                 if feature == word.lower():
#                     break
#                 position += 1